完全版は
https://adv-r.hadley.nz/

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Motivation" data-toc-modified-id="Motivation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Motivation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Generate-a-sample-dataset" data-toc-modified-id="Generate-a-sample-dataset-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Generate a sample dataset</a></span></li><li><span><a href="#lapply-is-equivalent-to-the-following" data-toc-modified-id="lapply-is-equivalent-to-the-following-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>lapply is equivalent to the following</a></span></li></ul></li><li><span><a href="#extra-argumentを使うのは必ずしもうまくいかない" data-toc-modified-id="extra-argumentを使うのは必ずしもうまくいかない-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>extra argumentを使うのは必ずしもうまくいかない</a></span></li><li><span><a href="#コードで似たような表現を繰り返すためにbugが起こりやすい例" data-toc-modified-id="コードで似たような表現を繰り返すためにbugが起こりやすい例-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>コードで似たような表現を繰り返すためにbugが起こりやすい例</a></span></li><li><span><a href="#以下のようにしたほうが良い" data-toc-modified-id="以下のようにしたほうが良い-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>以下のようにしたほうが良い</a></span></li><li><span><a href="#sandbox" data-toc-modified-id="sandbox-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>sandbox</a></span></li></ul></li><li><span><a href="#anonymous-function" data-toc-modified-id="anonymous-function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>anonymous function</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#is.numericをベクトルにかけると１個のboolのみが返る" data-toc-modified-id="is.numericをベクトルにかけると１個のboolのみが返る-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>is.numericをベクトルにかけると１個のboolのみが返る</a></span></li></ul></li><li><span><a href="#Like-all-functions-in-R,-anonymous-functions-have-formals(),-a-body(),-and-a-parent-environment()" data-toc-modified-id="Like-all-functions-in-R,-anonymous-functions-have-formals(),-a-body(),-and-a-parent-environment()-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Like all functions in R, anonymous functions have formals(), a body(), and a parent environment()</a></span></li></ul></li><li><span><a href="#Closure" data-toc-modified-id="Closure-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Closure</a></span></li></ul></div>

In [1]:
options(repos = structure(c(CRAN = "http://cran.ism.ac.jp/")))

# Motivation

### Generate a sample dataset

In [3]:
set.seed(1014)
df <- data.frame(replicate(6, sample(c(1:10, -99), 6, rep = TRUE)))
names(df) <- letters[1:6]
df

a,b,c,d,e,f
1,6,1,5,-99,1
10,4,4,-99,9,3
7,9,5,4,1,4
2,9,3,8,6,8
1,10,5,9,8,6
6,2,1,3,8,5


### lapply is equivalent to the following

In [10]:
my_lapply = function(x , f, ...) {
    out <- vector("list", length(x))
    for (i in seq_along(x)) {
      out[[i]] <- f(x[[i]], ...)
    }
    out
    }

In [7]:
fix_missing <- function(x) {
  x[x == -99] <- NA
  x
}

In [8]:
lapply(df , fix_missing)

$a
[1]  1 10  7  2  1  6

$b
[1]  6  4  9  9 10  2

$c
[1] 1 4 5 3 5 1

$d
[1]  5 NA  4  8  9  3

$e
[1] NA  9  1  6  8  8

$f
[1] 1 3 4 8 6 5

In [11]:
my_lapply(df , fix_missing)

[[1]]
[1]  1 10  7  2  1  6

[[2]]
[1]  6  4  9  9 10  2

[[3]]
[1] 1 4 5 3 5 1

[[4]]
[1]  5 NA  4  8  9  3

[[5]]
[1] NA  9  1  6  8  8

[[6]]
[1] 1 3 4 8 6 5

In [ ]:
missing_fixer <- function(na_value) {
  function(x) {
    x[x == na_value] <- NA
    x
  }
}
fix_missing_99 <- missing_fixer(-99)
fix_missing_999 <- missing_fixer(-999)

In [14]:
fix_missing_99(c(-99, -999))

[1]   NA -999

In [15]:
fix_missing_999(c(-99,-999))

[1] -99  NA

## extra argumentを使うのは必ずしもうまくいかない

extra argumentを使う例：

In [16]:
fix_missing <- function(x, na.value) {
  x[x == na.value] <- NA
  x
}

## コードで似たような表現を繰り返すためにbugが起こりやすい例

In [17]:
summary <- function(x) {
  c(mean(x), median(x), sd(x), mad(x), IQR(x))
}
lapply(df, summary)

$a
[1] 4.50000 4.00000 3.72827 4.44780 5.50000

$b
[1] 6.666667 7.500000 3.204164 2.965200 4.500000

$c
[1] 3.166667 3.500000 1.834848 2.223900 3.250000

$d
[1] -11.66667   4.50000  42.84701   3.70650   4.00000

$e
[1] -11.16667   7.00000  43.12501   2.22390   5.75000

$f
[1] 4.500000 4.500000 2.428992 2.223900 2.500000

In [18]:
summary <- function(x) {
 c(mean(x, na.rm = TRUE),
   median(x, na.rm = TRUE),
   sd(x, na.rm = TRUE),
   mad(x, na.rm = TRUE),
   IQR(x, na.rm = TRUE))
}

In [20]:
lapply(df , summary)

$a
[1] 4.50000 4.00000 3.72827 4.44780 5.50000

$b
[1] 6.666667 7.500000 3.204164 2.965200 4.500000

$c
[1] 3.166667 3.500000 1.834848 2.223900 3.250000

$d
[1] -11.66667   4.50000  42.84701   3.70650   4.00000

$e
[1] -11.16667   7.00000  43.12501   2.22390   5.75000

$f
[1] 4.500000 4.500000 2.428992 2.223900 2.500000

## 以下のようにしたほうが良い

In [22]:
summary <- function(x) {
  funs <- c(mean, median, sd, mad, IQR)
  lapply(funs, function(f) f(x, na.rm = TRUE))
}

In [27]:
summary(c(1,2))

[[1]]
[1] 1.5

[[2]]
[1] 1.5

[[3]]
[1] 0.7071068

[[4]]
[1] 0.7413

[[5]]
[1] 0.5

In [28]:
lapply(df, summary)

$a
$a[[1]]
[1] 4.5

$a[[2]]
[1] 4

$a[[3]]
[1] 3.72827

$a[[4]]
[1] 4.4478

$a[[5]]
[1] 5.5


$b
$b[[1]]
[1] 6.666667

$b[[2]]
[1] 7.5

$b[[3]]
[1] 3.204164

$b[[4]]
[1] 2.9652

$b[[5]]
[1] 4.5


$c
$c[[1]]
[1] 3.166667

$c[[2]]
[1] 3.5

$c[[3]]
[1] 1.834848

$c[[4]]
[1] 2.2239

$c[[5]]
[1] 3.25


$d
$d[[1]]
[1] -11.66667

$d[[2]]
[1] 4.5

$d[[3]]
[1] 42.84701

$d[[4]]
[1] 3.7065

$d[[5]]
[1] 4


$e
$e[[1]]
[1] -11.16667

$e[[2]]
[1] 7

$e[[3]]
[1] 43.12501

$e[[4]]
[1] 2.2239

$e[[5]]
[1] 5.75


$f
$f[[1]]
[1] 4.5

$f[[2]]
[1] 4.5

$f[[3]]
[1] 2.428992

$f[[4]]
[1] 2.2239

$f[[5]]
[1] 2.5

## sandbox

In [29]:
lapply(c(mean , sd , median) , 
       function(f){ 
           function(x) { 
               f(x , na.rm = T)
           }
       }
      )

[[1]]
function (x) 
{
    f(x, na.rm = T)
}
<environment: 0x0000000018ffddb8>

[[2]]
function (x) 
{
    f(x, na.rm = T)
}
<bytecode: 0x0000000018e95e68>
<environment: 0x0000000018ffe1a8>

[[3]]
function (x) 
{
    f(x, na.rm = T)
}
<bytecode: 0x0000000018e95e68>
<environment: 0x0000000018e8f8f8>

# anonymous function

The data was extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973–74 models).

In [30]:
mtcars

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


In [33]:
lapply(mtcars, function(x) length(unique(x)))   

$mpg
[1] 25

$cyl
[1] 3

$disp
[1] 27

$hp
[1] 22

$drat
[1] 22

$wt
[1] 29

$qsec
[1] 30

$vs
[1] 2

$am
[1] 2

$gear
[1] 3

$carb
[1] 6

In [36]:
lapply(mtcars , function(x) { length(na.omit(x))})

$mpg
[1] 32

$cyl
[1] 32

$disp
[1] 32

$hp
[1] 32

$drat
[1] 32

$wt
[1] 32

$qsec
[1] 32

$vs
[1] 32

$am
[1] 32

$gear
[1] 32

$carb
[1] 32

### is.numericをベクトルにかけると１個のboolのみが返る

In [48]:
is.numeric(c(1,2))

[1] TRUE

In [47]:
?is.numeric

In [44]:
Filter(function(x) { paste(x ,  collapse = "_") } , mtcars)

Mazda RX4
Mazda RX4 Wag
Datsun 710
Hornet 4 Drive
Hornet Sportabout
Valiant
Duster 360
Merc 240D
Merc 230
Merc 280
Merc 280C


In [41]:
Filter(function(x) !is.numeric(x), mtcars)  

Mazda RX4
Mazda RX4 Wag
Datsun 710
Hornet 4 Drive
Hornet Sportabout
Valiant
Duster 360
Merc 240D
Merc 230
Merc 280
Merc 280C


In [32]:
integrate(function(x) sin(x) ^ 2, 0, pi)

$mpg
[1] 25

$cyl
[1] 3

$disp
[1] 27

$hp
[1] 22

$drat
[1] 22

$wt
[1] 29

$qsec
[1] 30

$vs
[1] 2

$am
[1] 2

$gear
[1] 3

$carb
[1] 6

Mazda RX4
Mazda RX4 Wag
Datsun 710
Hornet 4 Drive
Hornet Sportabout
Valiant
Duster 360
Merc 240D
Merc 230
Merc 280
Merc 280C


1.570796 with absolute error < 1.7e-14

In [31]:
?mtcars

In [4]:
# Generate a sample dataset
set.seed(1014)
df <- data.frame(replicate(6, sample(c(1:10, -99), 6, rep = TRUE)))
names(df) <- letters[1:6]
df

a,b,c,d,e,f
1,6,1,5,-99,1
10,4,4,-99,9,3
7,9,5,4,1,4
2,9,3,8,6,8
1,10,5,9,8,6
6,2,1,3,8,5


In [6]:
replicate(6, sample(c(1:10, -99), 6, rep = TRUE))

8,8,-99,6,10,-99
-99,8,5,9,4,6
2,2,6,3,7,6
3,9,4,8,4,4
8,9,2,1,7,5
6,-99,6,4,3,5


In [7]:
fix_missing <- function(x) {
  x[x == -99] <- NA
  x
}
df[] <- lapply(df, fix_missing)

In [8]:
df

a,b,c,d,e,f
1,6,1,5,NA,1
10,4,4,NA,9,3
7,9,5,4,1,4
2,9,3,8,6,8
1,10,5,9,8,6
6,2,1,3,8,5


In [11]:
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [12]:
df$a %>% length


[1] 6

In [15]:
tmp = df

In [18]:
data.frame(lapply(df , length))

a,b,c,d,e,f
6,6,6,6,6,6


In [17]:
tmp

a,b,c,d,e,f
6,6,6,6,6,6
6,6,6,6,6,6
6,6,6,6,6,6
6,6,6,6,6,6
6,6,6,6,6,6
6,6,6,6,6,6


In [19]:
missing_fixer <- function(na_value) {
  function(x) {
    x[x == na_value] <- NA
    x
  }
}
fix_missing_99 <- missing_fixer(-99)
fix_missing_999 <- missing_fixer(-999)

fix_missing_99(c(-99, -999))

[1]   NA -999

## Like all functions in R, anonymous functions have formals(), a body(), and a parent environment()

In [50]:
formals(function(x = 4) g(x) + h(x))

$x
[1] 4


In [51]:
body(function(x = 4) g(x) + h(x))

g(x) + h(x)

In [52]:
environment(function(x = 4) g(x) + h(x))

<environment: R_GlobalEnv>

In [53]:
ls(envir = environment(function(x = 4) g(x) + h(x)))

[1] "df"              "fix_missing"     "fix_missing_99"  "fix_missing_999"
[5] "missing_fixer"   "my_lapply"       "summary"

# Closure

- closureはfunctionによって生成されるfunction
- anonymous functionはclosure生成時にも使われる

In [54]:
power <- function(exponent) {
  function(x) {
    x ^ exponent
  }
}

square <- power(2)
square(2)

[1] 4

In [55]:
quatro = power(4)

In [57]:
quatro(3)

[1] 81